<a href="https://colab.research.google.com/github/pablo-carmonam-um-es/pcd_boletin1/blob/main/pr/bd2-sesion0-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 1: Definición de esquemas


En esta hoja introduciremos la forma de trabajar con esquemas, que son fundamentales para trabajar correctamente con los datos. Aunque hay bases de datos NoSQL que no obligan a tener un esquema en muchos casos, podemos definirlos para mejorar la calidad de los datos y su tratamiento.

La siguiente celda diferencia entre Jupyter Notebook y Google Colab, ya que tienen pequeñas diferencias. Todos los Notebooks de la asignatura se pueden ejecutar en local usando Docker. Los detalles no los veremos aquí, pero los podéis preguntar al profesor.

In [6]:
import sys

RunningInCOLAB: bool = "google.colab" in sys.modules

Instalamos `pandas` y todas sus dependencias opcionales para mejorar el rendimiento, los gráficos, el formato de salida y el soporte de ficheros como `Parquet`.

In [7]:
%pip install -q --upgrade 'pandas[performance,parquet,plot,output-formatting,computation,html]'

A continuación mostramos los paquetes que usaremos regularmente para tratar datos, `pandas`, `numpy`, `matplotlib`. Al ser un programa en Python, se pueden importar los paquetes necesarios, y seguirán siendo válidos hasta el final del _notebook_.

In [8]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Lo siguiente hace que los gráficos se muestren inline. Para figuras pequeñas se puede utilizar unas figuras interactivas que permiten zoom, usando `%maplotlib nbagg`.

In [9]:
%matplotlib inline
matplotlib.style.use("ggplot")

## Importación inicial de los datos de Stackoverflow

El conjunto de datos de Stackoverflow es un *dump* de datos que cada cierto tiempo realiza el sitio web stackoverflow.com, en particular, la versión en español, http://es.stackoverflow.com. El formato de los datos es XML, aunque es muy sencillo de extraer los datos, como veremos a continuación.

El contenido original se puede descargar directamente de los diferentes _dumps_ que se realizan de la página de archive.org: https://archive.org/details/stackexchange.

Sin embargo, nosotros descargaremos una versión fija previamente descargada para que todos usemos los mismos datos.

### Formato de los datos de Stackoverflow ###

A continuación mostramos los ficheros y una breve explicación de los campos que contienen. Ten en cuenta que hemos puesto primero los ficheros que más usaremos a lo largo del curso.

**> Notas generales**
- Formato: cada fichero es XML y suele distribuirse comprimido en formato 7zipped.
- Codificación: UTF-8. Los campos de fecha siguen el formato ISO8601: "YYYY-MM-DDTHH:mm:ss.fff".
- Nulos: cuando un campo no aplica (p.ej., `ParentId` en preguntas), no aparece o viene vacío; en nuestros Parquet suele ser `null`.
- Identificadores: `Id` es clave primaria en cada entidad. Claves foráneas enlazan por `Id` entre ficheros.

---

Fichero: <b>posts.xml</b> (preguntas y respuestas)
- Id: Identificador de la publicación. Ej: 98765. Uso: PK.
- PostTypeId: Tipo (1=Pregunta, 2=Respuesta). Ej: 1. Uso: rutas de procesamiento/particionado.
- ParentId: Identificador de la pregunta padre si es respuesta. Ej: 1234. Uso: FK (Foreign Key) a `posts.Id` (solo si `PostTypeId=2`).
- AcceptedAnswerId: Id de la respuesta aceptada (solo en preguntas). Ej: 4567. Uso: FK a `posts.Id` para marcar aceptadas.
- CreationDate: Fecha de creación. Ej: "2010-02-15T10:21:34.120". Uso: análisis temporal.
- Score: Votos netos (upvotes-downvotes). Ej: 42. Uso: ranking/calidad.
- ViewCount: Nº de vistas (solo preguntas). Ej: 12345. Uso: popularidad.
- Body: HTML/Markdown del cuerpo. Ej: "<p>¿Cómo puedo leer ...?</p>". Uso: contenido/NLP.
- OwnerUserId: Autor. Ej: 271828. Uso: FK a `users.Id`.
- OwnerDisplayName: Nombre mostrado si el autor fue eliminado. Ej: "John Doe". Uso: se usa cuando `OwnerUserId` es nulo (usuario eliminado o anónimo).
- LastEditorUserId: Último usuario que editó. Ej: 314159. Uso: FK a `users.Id`.
- LastEditorDisplayName: Nombre si el editor fue eliminado. Ej: "Community". Uso: solo cuando `LastEditorUserId` es nulo.
- LastEditDate: Fecha de última edición. Ej: "2009-03-05T22:28:34.823". Uso: auditoría.
- LastActivityDate: Última actividad (edición, comentario, etc.). Ej: "2009-03-11T12:51:01.480". Uso: ordenación por actividad reciente.
- CommunityOwnedDate: Fecha desde la que es de la comunidad. Ej: "2009-03-11T12:51:01.480". Uso: gobernanza.
- ClosedDate: Fecha de cierre (si procede). Ej: "2012-01-01T12:00:00.000". Uso: moderación.
- Title: Título de la pregunta. Ej: "¿Cómo unir listas en Python?". Uso: metadata/búsqueda.
- Tags: Lista de etiquetas en formato XML/HTML: "<python><list><merge>". Uso: clasificación por tema.
- AnswerCount: Nº de respuestas (en preguntas). Ej: 5. Uso: engagement.
- CommentCount: Nº de comentarios. Ej: 3. Uso: actividad.
- FavoriteCount: Favoritos marcados (legacy). Ej: 10. Uso: popularidad histórica.
- ContentLicense: Licencia. Ej: "CC BY-SA 4.0". Uso: cumplimiento.

---

Fichero: <b>users.xml</b> (usuarios)
- Id: PK del usuario. Ej: 271828.
- AccountId: Id de cuenta en la red Stack Exchange. Ej: 123456. Uso: consolidación multi-sitio.
- Reputation: Reputación. Ej: 15234. Uso: privilegios/ordenación.
- CreationDate: Alta del usuario. Ej: "2008-07-31T21:42:52.667".
- DisplayName: Nombre mostrado. Ej: "Jane Doe". Uso: UI.
- EmailHash: Hash MD5 de email (legacy). Ej: "fcea920f...". Uso: avatar histórico.
- LastAccessDate: Último acceso. Ej: "2020-04-01T12:00:00.000". Uso: actividad.
- WebsiteUrl: Web personal. Ej: "https://janedoe.dev".
- Location: Ubicación libre. Ej: "Madrid, ES".
- Age: Edad (cuando se compartía). Ej: 32.
- AboutMe: BIO en HTML/Markdown. Ej: "<p>Desarrollo datos...</p>".
- Views: Visitas al perfil. Ej: 1234.
- UpVotes: Upvotes emitidos por el usuario. Ej: 500.
- DownVotes: Downvotes emitidos. Ej: 20.
- ProfileImageUrl: Avatar. Ej: "https://.../image.png".

---

Fichero: <b>votes.xml</b> (votos a publicaciones)
- Id: PK del voto. Ej: 7777.
- PostId: Publicación votada. Ej: 98765. Uso: FK a `posts.Id`.
- VoteTypeId: Tipo de voto/acción. Uso: métricas de calidad/moderación.
  - 1: AcceptedByOriginator – el autor de la pregunta acepta una respuesta (equivale a `AcceptedAnswerId`).
  - 2: UpMod – upvote.
  - 3: DownMod – downvote.
  - 4: Offensive – ofensivo (legacy).
  - 5: Favorite – marcado como favorito (legacy); si `VoteTypeId=5`, `UserId` se rellena.
  - 6: Close – voto de cierre.
  - 7: Reopen – voto de reapertura.
  - 8: BountyStart – inicio de recompensa.
  - 9: BountyClose – cierre de recompensa; si `VoteTypeId=9`, `BountyAmount` se rellena.
  - 10: Deletion – voto de borrado.
  - 11: Undeletion – voto de restauración.
  - 12: Spam – marcado como spam.
  - 13: InformModerator – informar a moderación.
- CreationDate: Fecha del voto. Ej: "2015-06-10T09:30:00.000".
- UserId: Usuario que vota (solo para algunos tipos como 5). Ej: 54321. Uso: FK a `users.Id`.
- BountyAmount: Cantidad de recompensa (solo tipo 9). Ej: 100.

---

Fichero: <b>tags.xml</b> (etiquetas del sitio)
- Id: PK de la etiqueta. Ej: 42.
- TagName: Nombre de la etiqueta. Ej: "python". Uso: clasificación temática.
- Count: Nº de usos en preguntas. Ej: 150234. Uso: popularidad.
- ExcerptPostId: Id del post con el extracto de la wiki de etiqueta. Ej: 123. Uso: FK a `posts.Id`.
- WikiPostId: Id del post con el artículo de la wiki de etiqueta. Ej: 124. Uso: FK a `posts.Id`.

---

Fichero: <b>badges.xml</b> (insignias logradas por usuarios)
- UserId: Id del usuario que recibe la insignia. Ej: 420. Uso: FK a `users.Id`.
- Name: Nombre de la insignia. Ej: "Teacher". Uso: clasificar tipo de logro.
- Date: Fecha de concesión. Ej: "2008-09-15T08:55:03.923". Uso: series temporales.
- Class: Categoría (1=Gold, 2=Silver, 3=Bronze). Ej: 3. Uso: nivel de la insignia.
- TagBased: Si la insignia es específica de etiqueta. Ej: true. Uso: filtrar logros por etiquetas.

---

Fichero: <b>comments.xml</b> (comentarios en publicaciones)
- Id: Identificador del comentario. Ej: 12345. Uso: PK.
- PostId: Publicación a la que comenta. Ej: 100234. Uso: FK a `posts.Id`.
- Score: Puntuación del comentario. Ej: 5. Uso: ordenar/filtrar relevancia.
- Text: Texto del comentario. Ej: "¿Puedes compartir el error exacto?". Uso: contenido.
- CreationDate: Fecha de creación. Ej: "2008-09-06T08:07:10.730". Uso: análisis temporal.
- UserId: Autor del comentario. Ej: 314159. Uso: FK a `users.Id`.
- UserDisplayName: Nombre mostrado si el usuario fue eliminado. Ej: "user123". Uso: se usa cuando `UserId` es nulo/ausente.
- ContentLicense: Licencia del contenido. Ej: "CC BY-SA 4.0". Uso: cumplimiento/licencias.


---

Fichero: <b>posthistory.xml</b> (historial detallado de cambios)
- Id: PK del evento de historial. Ej: 5555.
- PostHistoryTypeId: Tipo de cambio. Uso: clasifica el evento. Ejemplos:
  - 1: Initial Title – primer título de la pregunta.
  - 2: Initial Body – primer cuerpo en crudo.
  - 3: Initial Tags – primeras etiquetas.
  - 4: Edit Title – cambio de título.
  - 5: Edit Body – cambio de cuerpo (markdown crudo).
  - 6: Edit Tags – cambio de etiquetas.
  - 7/8/9: Rollback Title/Body/Tags – revertidos.
  - 10: Post Closed – cierre por votos.
  - 11: Post Reopened – reapertura.
  - 12/13: Post Deleted/Undeleted – borrado/restaurado.
  - 14/15: Post Locked/Unlocked – bloqueado/desbloqueado.
  - 16: Community Owned – pasa a comunidad.
  - 17: Post Migrated – migración (origen/destino).
  - 18: Question Merged – fusión de preguntas.
  - 19/20: Question Protected/Unprotected – protegido/desprotegido.
  - 21: Post Disassociated – se elimina el OwnerUserId.
  - 22: Question Unmerged – deshace fusión.
- PostId: Publicación afectada. Ej: 98765. Uso: FK a `posts.Id`.
- RevisionGUID: Agrupa múltiples registros de un mismo acto. Ej: "3E5B...". Uso: correlación.
- CreationDate: Fecha del evento. Ej: "2009-03-05T22:28:34.823".
- UserId: Usuario que realiza el cambio. Ej: 271828. Uso: FK a `users.Id`.
- UserDisplayName: Si el usuario fue eliminado. Ej: "user123". Uso: cuando `UserId` nulo.
- Comment: Comentario del editor. Ej: "typo fix". Uso: auditoría.
- Text: Valor crudo nuevo asociado al cambio. Ej: nuevo markdown o JSON.
  - Para tipos 10–15: JSON con usuarios que han votado esa acción de moderación.
  - Para tipo 17: Detalles de migración: "from <url>" o "to <url>".
- CloseReasonId: Motivo de cierre (histórico). Ejemplos:
  - 1: Exact Duplicate – duplicada de otra.
  - 2: off-topic – fuera de tema.
  - 3: subjective – demasiado subjetiva.
  - 4: not a real question – no es una pregunta real.
  - 7: too localized – demasiado localizada.

---

Fichero: <b>postlinks.xml</b> (enlaces entre publicaciones)
- Id: PK del enlace. Ej: 2222.
- CreationDate: Fecha del enlace. Ej: "2011-05-05T10:00:00.000".
- PostId: Publicación origen. Ej: 1000. Uso: FK a `posts.Id`.
- RelatedPostId: Publicación destino. Ej: 1001. Uso: FK a `posts.Id`.
- PostLinkTypeId: Tipo de relación. Uso: grafo entre posts.
  - 1: Linked – relacionados.
  - 3: Duplicate – duplicados.

---


### Descarga de los datos

En nuestro caso los datos de Stackoverflow que vamos a usar están disponibles en un repositorio git de la asignatura. Con las siguientes instrucciones descargamos, sino se ha hecho ya, los ficheros y los descomprimimos.

In [10]:
%%sh
# No descargar los datos si ya existen
test -e Posts.xml || \
  (curl -fsL https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.xml.tar.xz.00;
   curl -fsL https://github.com/dsevilla/bd2-data/raw/main/es.stackoverflow/es.stackoverflow.xml.tar.xz.01) |\
     tar Jxvf -

Realizamos un `ls` para listar los ficheros descargados y comprobar que se han descargado correctamente.

In [11]:
!ls -lh *.xml

-rw-r--r-- 1 1000 users 206M Dec  4  2023 Comments.xml
-rw-r--r-- 1 1000 users 983M Dec  4  2023 Posts.xml
-rw-r--r-- 1 1000 users 223K Dec  4  2023 Tags.xml
-rw-r--r-- 1 1000 users  73M Dec  4  2023 Users.xml
-rw-r--r-- 1 1000 users  70M Dec  4  2023 Votes.xml


### Inspección y procesado

Podemos inspeccionar los ficheros `.xml` para ver su contenido. Son XML, sí, pero ¿con qué formato? A continuación trabajaremos con el fichero **Posts.xml**.

In [12]:
!head Posts.xml

﻿<?xml version="1.0" encoding="utf-8"?>
<posts>
  <row Id="1" PostTypeId="1" AcceptedAnswerId="2" CreationDate="2015-10-29T15:56:52.933" Score="40" ViewCount="780" Body="&lt;p&gt;Estoy creando un servicio usando &lt;em&gt;ASP.NET WebApi&lt;/em&gt;. Quiero añadir soporte para la negociación del tipo de contenido basado en extensiones en el &lt;em&gt;URI&lt;/em&gt;, así que he añadido lo siguiente al código de inicialización del servicio:&lt;/p&gt;&#xA;&#xA;&lt;pre&gt;&lt;code&gt;public static class WebApiConfig&#xA;{&#xA;  public static void Register(HttpConfiguration config)&#xA;  {&#xA;    config.Formatters.JsonFormatter.AddUriPathExtensionMapping(&quot;json&quot;, &quot;application/json&quot;);&#xA;    config.Formatters.XmlFormatter.AddUriPathExtensionMapping(&quot;xml&quot;, &quot;application/xml&quot;);&#xA;  }&#xA;}&#xA;&lt;/code&gt;&lt;/pre&gt;&#xA;&#xA;&lt;p&gt;Para que esto funcione necesito crear dos rutas para cada acción del controlador (estoy usando exclusivamente enrutamie

Se puede procesar el formato XML, y lo que podemos ver es que cada entrada es exactamente una línea que comienza por «`  <row`», y que contiene un conjunto de atributos en formato «`atributo="valor"`», donde valor va siempre entre comillas dobles. Si lo comprobamos, no existe ninguna comilla doble **dentro** de otra comilla doble, así que podemos extraer esos pares de forma fácil.

Como se ha visto en la teoría, se puede utilizar pandas `read_xml()` para leer directamente ficheros XML. A continuación se muestra un ejemplo de cómo leer el fichero `Posts.xml`.

**OJO**: Esto se hace por conveniencia, pero es posible que el fichero XML no quepa en memoria y Pandas daría error. Habría entonces que procesarlo como vimos en clase con el API SAX.

In [13]:
from pandas import DataFrame

posts_df: DataFrame = pd.read_xml('Posts.xml', xpath='.//row')
posts_df.head()

,Id,PostTypeId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorDisplayName,LastEditDate,...,Tags,AnswerCount,CommentCount,ContentLicense,ParentId,LastEditorUserId,OwnerDisplayName,CommunityOwnedDate,ClosedDate,FavoriteCount
0,1,1,2.0,2015-10-29T15:56:52.933,40,780.0,<p>Estoy creando un servicio usando <em>ASP.NE...,23.0,user13558,2019-07-07T21:36:17.737,...,<.net><asp.net-web-api><asp.net>,1.0,2,CC BY-SA 4.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2,NaN,2015-10-29T19:14:23.673,31,NaN,<p>He encontrado la solución.</p>\n\n<p>Result...,23.0,NaN,NaN,...,NaN,NaN,2,CC BY-SA 3.0,1.0,NaN,NaN,NaN,NaN,NaN
2,3,1,9.0,2015-10-29T23:54:31.947,20,1035.0,"<p>Luego de ver cierto código, me he dado cuen...",21.0,NaN,2015-12-03T16:24:56.370,...,<delphi>,3.0,1,CC BY-SA 3.0,NaN,20.0,NaN,NaN,NaN,NaN
3,4,2,NaN,2015-10-30T00:45:47.640,6,NaN,"<p><code>.AsString</code> devuelve el mismo ""<...",24.0,NaN,2015-12-14T00:58:15.613,...,NaN,NaN,0,CC BY-SA 3.0,3.0,25.0,NaN,NaN,NaN,NaN
4,5,1,208.0,2015-10-30T01:15:27.267,37,37867.0,<p>¿Cuál es la forma más eficiente de separar ...,24.0,NaN,2016-09-27T17:46:24.900,...,<c++><string>,7.0,1,CC BY-SA 3.0,NaN,729.0,NaN,NaN,NaN,NaN


Ajustamos los tipos de datos para optimizar memoria y rendimiento.

In [14]:
posts_df = posts_df.convert_dtypes()

In [15]:
posts_df.describe(include='all')

,Id,PostTypeId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,LastEditorDisplayName,LastEditDate,...,Tags,AnswerCount,CommentCount,ContentLicense,ParentId,LastEditorUserId,OwnerDisplayName,CommunityOwnedDate,ClosedDate,FavoriteCount
count,410346.0,410346.0,82491.0,410346,410346.0,194788.0,409801,403879.0,7325,183462,...,194788,194788.0,410346.0,410346,211615.0,176290.0,6535,358,8191,4263.0
unique,<NA>,<NA>,<NA>,407635,<NA>,<NA>,409724,<NA>,358,178241,...,68754,<NA>,<NA>,2,<NA>,<NA>,882,343,8191,<NA>
top,<NA>,<NA>,<NA>,2015-10-30T16:53:47.187,<NA>,<NA>,<p>No es la opción más elegante (se que existe...,<NA>,user13558,2020-06-11T10:54:57.430,...,<javascript>,<NA>,<NA>,CC BY-SA 4.0,<NA>,<NA>,user75901,2018-03-06T18:59:54.457,2020-07-23T22:32:14.130,<NA>
freq,<NA>,<NA>,<NA>,2,<NA>,<NA>,4,<NA>,1592,3056,...,4836,<NA>,<NA>,297244,<NA>,<NA>,641,6,1,<NA>
mean,297087.937341,1.549439,283647.713472,NaN,0.909981,1163.573932,NaN,102858.213465,NaN,NaN,...,NaN,1.086386,1.68085,NaN,275834.698448,77005.894248,NaN,NaN,NaN,0.000235
std,178509.086993,0.57867,173429.392114,NaN,2.193521,5428.851117,NaN,89903.902647,NaN,NaN,...,NaN,0.869047,2.397965,NaN,175858.4523,83144.237384,NaN,NaN,NaN,0.015316
min,1.0,1.0,2.0,NaN,-28.0,2.0,NaN,-1.0,NaN,NaN,...,NaN,0.0,0.0,NaN,1.0,-1.0,NaN,NaN,NaN,0.0
25%,140770.25,1.0,131793.0,NaN,0.0,72.0,NaN,25463.0,NaN,NaN,...,NaN,1.0,0.0,NaN,120381.5,6798.0,NaN,NaN,NaN,0.0
50%,293154.0,2.0,276044.0,NaN,0.0,209.0,NaN,81357.0,NaN,NaN,...,NaN,1.0,1.0,NaN,265276.0,44032.0,NaN,NaN,NaN,0.0
75%,449105.5,2.0,428137.0,NaN,1.0,704.0,NaN,159545.0,NaN,NaN,...,NaN,1.0,2.0,NaN,421343.0,128299.0,NaN,NaN,NaN,0.0


In [16]:
posts_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 410346 entries, 0 to 410345
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   Id                     410346 non-null  Int64 
 1   PostTypeId             410346 non-null  Int64 
 2   AcceptedAnswerId       82491 non-null   Int64 
 3   CreationDate           410346 non-null  string
 4   Score                  410346 non-null  Int64 
 5   ViewCount              194788 non-null  Int64 
 6   Body                   409801 non-null  string
 7   OwnerUserId            403879 non-null  Int64 
 8   LastEditorDisplayName  7325 non-null    string
 9   LastEditDate           183462 non-null  string
 10  LastActivityDate       410346 non-null  string
 11  Title                  194788 non-null  string
 12  Tags                   194788 non-null  string
 13  AnswerCount            194788 non-null  Int64 
 14  CommentCount           410346 non-null  Int64 
 15  ContentLice

Otra modificación que hay que hacer es cambiar los retornos de carro y saltos de línea por cambios de HTML, es decir, por `<br>`, para evitar que, al exportar a CSV, se generen múltiples líneas por cada entrada. Esto se puede hacer con una función sencilla de reemplazo.


In [17]:
import re

posts_df['Body'] = posts_df['Body'].apply(
    lambda s: re.sub(r'\r*\n', '<br/>', s) if pd.notnull(s) else s
)

El conjunto de atributos del fichero **Post.xml** es:

In [18]:
attr_set: set[str] = set(posts_df.columns.to_list())
attr_set

{'AcceptedAnswerId',
 'AnswerCount',
 'Body',
 'ClosedDate',
 'CommentCount',
 'CommunityOwnedDate',
 'ContentLicense',
 'CreationDate',
 'FavoriteCount',
 'Id',
 'LastActivityDate',
 'LastEditDate',
 'LastEditorDisplayName',
 'LastEditorUserId',
 'OwnerDisplayName',
 'OwnerUserId',
 'ParentId',
 'PostTypeId',
 'Score',
 'Tags',
 'Title',
 'ViewCount'}

Como sabemos que el atributo `Id` va a ser la clave primaria, lo ponemos al principio. Además, generamos una lista, no un conjunto, para que el orden sea conocido.

In [19]:
attr_set.remove("Id")
all_attrs: list[str] = sorted(attr_set)
all_attrs.insert(0, "Id")
all_attrs

['Id',
 'AcceptedAnswerId',
 'AnswerCount',
 'Body',
 'ClosedDate',
 'CommentCount',
 'CommunityOwnedDate',
 'ContentLicense',
 'CreationDate',
 'FavoriteCount',
 'LastActivityDate',
 'LastEditDate',
 'LastEditorDisplayName',
 'LastEditorUserId',
 'OwnerDisplayName',
 'OwnerUserId',
 'ParentId',
 'PostTypeId',
 'Score',
 'Tags',
 'Title',
 'ViewCount']

### Formatos CSV, JSON y Parquet

En este boletín trabajamos con tres formatos de ficheros:


* CSV (Comma-Separated Values): formato de texto plano donde los datos se   organizan en filas y columnas separadas por comas. Se usa mucho para tablas simples.

* JSON (JavaScript Object Notation): formato de texto estructurado en pares clave–valor. Este formado es muy usado para representar datos jerárquicos o semiestructurados.

* Parquet: formato columnar optimizado para almacenamiento y lectura eficiente en grandes volúmenes de datos. Este formato es muy usado en entornos de Big Data.

Antes de continuar, vamos a ver cómo sería el formato interno de estos tres tipos de ficheros. Usamos de ejemplo un fichero fichero que tendría seis columnas: Id, Nombre, Apellidos, Fecha nacimiento, Grado y Curso. Todos almacenarían un total de 4 elementos.

Ejemplo de un fichero en formato **CSV**:

```csv
id,nombre,apellidos,fecha_nacimiento,grado,curso
1,Ana,García López,2003-05-12,Ingeniería Informática,3º
2,Carlos,Pérez Martín,2002-11-03,Ciencia e Ingeniería de Datos,4º
3,Lucía,Sánchez Díaz,2004-02-24,Ingeniería Informática,2º
4,David,Ruiz Ortega,2001-09-15,Ciencia e Ingeniería de Datos,1º
```


Ejemplo del mismo fichero en formato **JSON**:

```json
[
  {
    "id": 1,
    "nombre": "Ana",
    "apellidos": "García López",
    "fecha_nacimiento": "2003-05-12",
    "grado": "Ingeniería Informática",
    "curso": "3º"
  },
  {
    "id": 2,
    "nombre": "Carlos",
    "apellidos": "Pérez Martín",
    "fecha_nacimiento": "2002-11-03",
    "grado": "Ciencia e Ingeniería de Datos",
    "curso": "4º"
  },
  {
    "id": 3,
    "nombre": "Lucía",
    "apellidos": "Sánchez Díaz",
    "fecha_nacimiento": "2004-02-24",
    "grado": "Ingeniería Informática",
    "curso": "2º"
  },
  {
    "id": 4,
    "nombre": "David",
    "apellidos": "Ruiz Ortega",
    "fecha_nacimiento": "2001-09-15",
    "grado": "Ciencia e Ingeniería de Datos",
    "curso": "1º"
  },
]
```


Ejemplo del mismo fichero en formato **Parquet**, en el hipotético caso de que el fichero se guardase en formato texto y no en binario:

```bash
PARQUET FILE - estudiantes.parquet
VERSION: 1.0
ROWS: 6
COLUMNS: 4
SCHEMA:
  id                INT64
  nombre            STRING
  apellidos         STRING
  fecha_nacimiento  DATE
  grado             STRING
  curso             STRING

COLUMNAR STORAGE:
COLUMN: id
[1, 2, 3, 4]
COLUMN: nombre
["Ana", "Carlos", "Lucía", "David"]
COLUMN: apellidos
["García López", "Pérez Martín", "Sánchez Díaz",
 "Ruiz Ortega"]
COLUMN: fecha_nacimiento
["2003-05-12", "2002-11-03", "2004-02-24",
 "2001-09-15"]
COLUMN: grado
["Ingeniería Informática", "Ciencia e Ingeniería de Datos",
 "Ingeniería Informática", "Ciencia e Ingeniería de Datos"]
COLUMN: curso
["3º", "4º", "2º", "1º"]
METADATA:
  CREATOR: pandas/pyarrow
  CREATED_AT: 2025-11-13T12:00:00Z
  ROW_GROUPS: 1
  COMPRESSION: SNAPPY
```

Adicionalmente al formato **JSON**, está el formato **JSON Lines**, donde cada línea es un objeto JSON independiente. Aquí se muestra cómo quedaría:


```json
{"id": 1, "nombre": "Ana", "apellidos": "García López", "fecha_nacimiento": "2003-05-12", "grado": "Ingeniería Informática", "curso": "3º"}
{"id": 2, "nombre": "Carlos", "apellidos": "Pérez Martín", "fecha_nacimiento": "2002-11-03", "grado": "Ciencia e Ingeniería de Datos", "curso": "4º"}
{"id": 3, "nombre": "Lucía", "apellidos": "Sánchez Díaz", "fecha_nacimiento": "2004-02-24", "grado": "Ingeniería Informática", "curso": "2º"}
{"id": 4, "nombre": "David", "apellidos": "Ruiz Ortega", "fecha_nacimiento": "2001-09-15", "grado": "Ciencia e Ingeniería de Datos", "curso": "1º"}
{"id": 5, "nombre": "Elena", "apellidos": "Morales Vega", "fecha_nacimiento": "2003-12-01", "grado": "Ingeniería Informática", "curso": "3º"}
{"id": 6, "nombre": "Marcos", "apellidos": "Fernández Soto", "fecha_nacimiento": "2002-07-19", "grado": "Ciencia e Ingeniería de Datos", "curso": "4º"}

```


### Escritura del formato CSV

El formato CSV está especificado en el estándar RFC 4180. https://www.ietf.org/rfc/rfc4180.txt. En general se puede utilizar la biblioteca `csv` de Python 3 y vamos a exportar una línea de cabecera con todos los campos. https://docs.python.org/3/library/csv.html.

Tendremos en cuenta que todas las filas tienen que tener las mismas columnas y en el mismo orden dado por `all_attrs`.

Aquí ya empezamos a tener el problema de cómo codificar las filas sin valores. ¿Se utiliza algún valor nulo? ¿cuál? ¿Se utiliza un valor vacío? ¿El valor vacío significa siempre 0 en numérico? ¿Y una cadena es vacía o no tiene valor? Es por esto que las bases de datos realmente guardan datos de tipo ternario:

- **Nulo**: no existe el valor, no se ha introducido.
- **Vacío**: se ha introducido un valor, pero es una cadena vacía o 0 o un valor por defecto.
- **Con valor**: se ha introducido un valor, y es un valor válido.

¿Qué decisión se toma? No es sencillo y depende del dominio.

Aquí ya empezamos a ver la importancia de un esquema. No es lo mismo dejar un campo vacío si es una fecha, una cadena de caracteres o un valor numérico.

Lo primero que vamos a construir es un iterador que, una vez conocidos todos los atributos, produce una fila con todos los atributos, y para los vacíos, un valor vacío.

In [20]:
import csv
from collections.abc import Iterable


def write_csv(destfile: str, all_attrs: list[str], iterator: Iterable[dict[str, str]]) -> None:
    with open(destfile, "w") as wf:
        cw: csv.DictWriter[str] = csv.DictWriter(wf, fieldnames=all_attrs, dialect="excel")

        # Escribir la línea de cabecera
        cw.writeheader()
        cw.writerows(iterator)

In [21]:
write_csv("Posts.csv", all_attrs, posts_df.to_dict(orient="records"))

In [22]:
!head Posts.csv

Id,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,ContentLicense,CreationDate,FavoriteCount,LastActivityDate,LastEditDate,LastEditorDisplayName,LastEditorUserId,OwnerDisplayName,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount
1,2,1,"<p>Estoy creando un servicio usando <em>ASP.NET WebApi</em>. Quiero añadir soporte para la negociación del tipo de contenido basado en extensiones en el <em>URI</em>, así que he añadido lo siguiente al código de inicialización del servicio:</p><br/><br/><pre><code>public static class WebApiConfig<br/>{<br/>  public static void Register(HttpConfiguration config)<br/>  {<br/>    config.Formatters.JsonFormatter.AddUriPathExtensionMapping(""json"", ""application/json"");<br/>    config.Formatters.XmlFormatter.AddUriPathExtensionMapping(""xml"", ""application/xml"");<br/>  }<br/>}<br/></code></pre><br/><br/><p>Para que esto funcione necesito crear dos rutas para cada acción del controlador (estoy usando exclusivamente enr

## Conversión hacia JSON

El siguiente código convierte el fichero CSV en al formato JSON que podéis ver en: https://www.json.org/json-en.html. El código funciona, pero tiene el problema de que para convertir todo a JSON, se tiene que generar un objeto (diccionario) JSON con **todos** los datos, que se tiene que almacenar en memoria. Esto no es siempre posible. Después veremos otro formato que no tiene este problema.

In [23]:
import json


def csv_to_json(fname_csv: str, fname_json: str, primary_key: str) -> None:
    data_dict: dict[str,dict] = {}

    with open(fname_csv, 'r', encoding='utf-8') as f_csv:
        csv_reader: csv.DictReader[str] = csv.DictReader(f_csv)

        for row in csv_reader:
            key: str = row[primary_key]
            data_dict[key] = row

    with open(fname_json, 'w', encoding='utf-8') as f_json:
        f_json.write(json.dumps(data_dict, indent=4, ensure_ascii=False))

In [24]:
fname_csv = 'Posts.csv'
fname_json = 'Posts.json'

csv_to_json(fname_csv, fname_json, 'Id')

In [25]:
!head -n 30 Posts.json

{
    "1": {
        "Id": "1",
        "AcceptedAnswerId": "2",
        "AnswerCount": "1",
        "Body": "<p>Estoy creando un servicio usando <em>ASP.NET WebApi</em>. Quiero añadir soporte para la negociación del tipo de contenido basado en extensiones en el <em>URI</em>, así que he añadido lo siguiente al código de inicialización del servicio:</p><br/><br/><pre><code>public static class WebApiConfig<br/>{<br/>  public static void Register(HttpConfiguration config)<br/>  {<br/>    config.Formatters.JsonFormatter.AddUriPathExtensionMapping(\"json\", \"application/json\");<br/>    config.Formatters.XmlFormatter.AddUriPathExtensionMapping(\"xml\", \"application/xml\");<br/>  }<br/>}<br/></code></pre><br/><br/><p>Para que esto funcione necesito crear dos rutas para cada acción del controlador (estoy usando exclusivamente enrutamiento basado en atributos):</p><br/><br/><pre><code>[Route(\"item/{id}/details\")]<br/>[Route(\"item/{id}/details.{ext}\")]<br/>[HttpGet]<br/>public ItemDetail[

Si nos damos cuenta, tenemos el problema de que el valor Id está por duplicado, ya que nos aparece como dos veces, una como clave del objeto y otra como el valor del campo `Id`.

Vamos a ver cómo eliminar columnas que no queramos tener.


In [26]:
def csv_to_json2(fname_csv: str, fname_json: str, primary_key: str) -> None:
    data_dict: dict[str, dict] = {}

    with open(fname_csv, 'r', encoding='utf-8') as f_csv:
        csv_reader: csv.DictReader[str] = csv.DictReader(f_csv)

        for rows in csv_reader:
            key: str = rows[primary_key]

            # Borramos los campos que nos interesen.
            del rows[primary_key]

            data_dict[key] = rows

    with open(fname_json, 'w', encoding='utf-8') as f_json:
        f_json.write(json.dumps(data_dict, indent=4, ensure_ascii=False))

In [27]:
fname_csv = 'Posts.csv'
fname_json = 'Posts.json'

csv_to_json2(fname_csv, fname_json, 'Id')

In [28]:
!head -n 100 Posts.json

{
    "1": {
        "AcceptedAnswerId": "2",
        "AnswerCount": "1",
        "Body": "<p>Estoy creando un servicio usando <em>ASP.NET WebApi</em>. Quiero añadir soporte para la negociación del tipo de contenido basado en extensiones en el <em>URI</em>, así que he añadido lo siguiente al código de inicialización del servicio:</p><br/><br/><pre><code>public static class WebApiConfig<br/>{<br/>  public static void Register(HttpConfiguration config)<br/>  {<br/>    config.Formatters.JsonFormatter.AddUriPathExtensionMapping(\"json\", \"application/json\");<br/>    config.Formatters.XmlFormatter.AddUriPathExtensionMapping(\"xml\", \"application/xml\");<br/>  }<br/>}<br/></code></pre><br/><br/><p>Para que esto funcione necesito crear dos rutas para cada acción del controlador (estoy usando exclusivamente enrutamiento basado en atributos):</p><br/><br/><pre><code>[Route(\"item/{id}/details\")]<br/>[Route(\"item/{id}/details.{ext}\")]<br/>[HttpGet]<br/>public ItemDetail[] GetItemDetails(in

Al escribir en formato JSON se nos queda un fichero compacto que no podemos dividir.

## JSON Lines

El problema es que el fichero JSON generado es muy grande, se puede decir que gigante, y hay que leerlo en memoria antes de procesarlo. Para evitar este problema, se creó el formato JSON Lines. En vez de tener un array que incluya a todo el fichero, se obliga a que cada objeto JSON incluido en el fichero esté en su propia línea.

Si algún elemento del JSON contiene un salto de línea, se codifica de alguna forma, como por ejemplo como `'\n'`. De esta forma ya están los datos en CSV, así que la conversión no será problemática.

Más información: https://jsonlines.org.

In [29]:
import json


def csv_to_jsonl(fname_csv, fname_jsonl):
    with open(fname_csv, 'r', encoding='utf-8') as f_csv:
        csv_reader: csv.DictReader[str] = csv.DictReader(f_csv)

        with open(fname_jsonl, 'w', encoding='utf-8') as f_jsonl:
            for row in csv_reader:
                json_line: str = json.dumps(row, ensure_ascii=False)
                f_jsonl.write(json_line)
                f_jsonl.write("\n")

In [30]:
csv_to_jsonl('Posts.csv', 'Posts.jsonl')

In [31]:
!head Posts.jsonl

{"Id": "1", "AcceptedAnswerId": "2", "AnswerCount": "1", "Body": "<p>Estoy creando un servicio usando <em>ASP.NET WebApi</em>. Quiero añadir soporte para la negociación del tipo de contenido basado en extensiones en el <em>URI</em>, así que he añadido lo siguiente al código de inicialización del servicio:</p><br/><br/><pre><code>public static class WebApiConfig<br/>{<br/>  public static void Register(HttpConfiguration config)<br/>  {<br/>    config.Formatters.JsonFormatter.AddUriPathExtensionMapping(\"json\", \"application/json\");<br/>    config.Formatters.XmlFormatter.AddUriPathExtensionMapping(\"xml\", \"application/xml\");<br/>  }<br/>}<br/></code></pre><br/><br/><p>Para que esto funcione necesito crear dos rutas para cada acción del controlador (estoy usando exclusivamente enrutamiento basado en atributos):</p><br/><br/><pre><code>[Route(\"item/{id}/details\")]<br/>[Route(\"item/{id}/details.{ext}\")]<br/>[HttpGet]<br/>public ItemDetail[] GetItemDetails(int id)<br/>{<br/>  return 

## Esquema para los datos

A continuación construiremos un esquema para estos datos. Esto se puede lograr infiriéndolo de los datos. En este caso lo haremos a mano, simulando el momento en el que se diseñó el modelo de datos original que dio lugar a la base de datos de donde se obtuvieron los mismos.

Para crear el esquema utilizamos `pyarrow`. Los esquemas de PyArrow son descripciones de la estructura de los datos, incluyendo los nombres de las columnas y sus tipos de datos. Su definición es muy sencilla, y en general sólo definen lo que se encesita, con el tipo `Schema`, y los elementos que lo componen, con su nombre, una indicación de si el campo es *nullable* (es decir, puede contener NULL), y su tipo de dato. También unos metadatos que se pueden añadir.

Los tipos de datos pueden ser:

- `pa.int32()`: Entero de 32 bits
- `pa.int64()`: Entero de 64 bits
- `pa.float32()`: Flotante de 32 bits
- `pa.float64()`: Flotante de 64 bits
- `pa.string()`: Cadena de texto
- `pa.binary()`: Datos binarios
- `pa.bool_()`: Valor booleano
- `pa.date32()`: Fecha (sin hora)
- `pa.date64()`: Fecha y hora
- `pa.timestamp()`: Marca de tiempo (fecha y hora).
- `pa.decimal()`: Número decimal de precisión arbitraria.
- `pa.list()`: Lista de valores.
- `pa.map()`: Mapa de claves y valores.
- `pa.struct()`: Estructura de campos anidados (esto permite representar datos complejos).

A continuación se define un esquema para los Posts:

In [32]:
# Añadir un esquema de pyarrow para todos los atributos
import pyarrow as pa
from pyarrow import Schema

posts_schema: Schema = pa.schema(
    [
        pa.field(
            "Id",
            pa.int64(),
            nullable=False,
            metadata={"primary key": "true", "description": "Unique identifier for the post"},
        ),
        pa.field(
            "AcceptedAnswerId",
            pa.int64(),
            metadata={
                "description": "Unique identifier for the accepted answer",
                "foreign key": "Posts.Id",
            },
        ),
        pa.field(
            "AnswerCount",
            pa.int32(),
            metadata={"description": "Number of answers to the post", "default": "0"},
        ),
        pa.field(
            "Body",
            pa.string(),
            metadata={"description": "Body content of the post (compressed, binary)"},
        ),
        pa.field(
            "ClosedDate",
            pa.timestamp("ms"),
            metadata={"description": "Date the post was closed"}
        ),
        pa.field(
            "CommentCount",
            pa.int32(),
            metadata={"description": "Number of comments on the post", "default": "0"},
        ),
        pa.field(
            "CommunityOwnedDate",
            pa.timestamp("ms"),
            metadata={"description": "Date the post was community owned"},
        ),
        pa.field(
            "ContentLicense",
            pa.string(),
            metadata={"description": "License for the content"}
        ),
        pa.field(
            "CreationDate",
            pa.timestamp("ms"),
            metadata={"description": "Date the post was created"},
        ),
        pa.field(
            "FavoriteCount",
            pa.int32(),
            metadata={"description": "Number of users who favorited the post", "default": "0"},
        ),
        pa.field(
            "LastActivityDate",
            pa.timestamp("ms"),
            metadata={"description": "Date of the last activity on the post"},
        ),
        pa.field(
            "LastEditDate",
            pa.timestamp("ms"),
            metadata={"description": "Date the post was last edited"},
        ),
        pa.field(
            "LastEditorDisplayName",
            pa.string(),
            metadata={
                "description": "Display name of the last editor (only if LastEditorUserId is not set)"
            },
        ),
        pa.field(
            "LastEditorUserId",
            pa.int64(),
            metadata={"description": "User ID of the last editor", "foreign key": "Users.Id"},
        ),
        pa.field(
            "OwnerDisplayName",
            pa.string(),
            metadata={"description": "Display name of the owner (only if OwnerUserId is not set)"},
        ),
        pa.field(
            "OwnerUserId",
            pa.int64(),
            metadata={
                "description": "Unique identifier for the user owner of the post. Foreign key to Users table",
                "foreign key": "Users.Id",
            },
        ),
        pa.field(
            "ParentId",
            pa.int64(),
            metadata={
                "description": "Unique identifier for the parent post, only in answers",
                "foreign key": "Posts.Id",
            },
        ),
        pa.field(
            "PostTypeId",
            pa.int64(),
            nullable=False,
            metadata={"description": "Type of the post: 1 (Question), 2 (Answer)"},
        ),
        pa.field(
            "Score",
            pa.int32(),
            metadata={
                "description": "Score of the post (plus votes minus less votes)",
                "default": "0",
            },
        ),
        pa.field(
            "Tags",
            pa.string(),
            metadata={"description": "List of tags in the form <tag1><tag2>..."},
        ),
        pa.field(
            "Title",
            pa.string(),
            metadata={"description": "Title of the post, only in questions"}
        ),
        pa.field(
            "ViewCount",
            pa.int64(),
            metadata={"description": "Number of views of the post", "default": "0"},
        ),
    ]
)

Ahora ya tenemos todas las filas que formarán parte de la tabla. Intentamos construir la tabla PyArrow con el esquema definido. Utilizamos la función de lectura de CSV de PyArrow para cargar los datos en la tabla. Podríamos haberlo hecho a través de la función `pa.Table.from_pandas()` si tuviéramos un DataFrame de Pandas, o de otra forma, pero lo importante e interesante es que hacemos que la tabla resultante (`posts_arrow`) tenga el esquema que hemos definido (`posts_schema`).


In [33]:
import pyarrow as pa
import pyarrow.csv as pacsv

posts_arrow: pa.Table = pacsv.read_csv(
    "Posts.csv",
    convert_options=pacsv.ConvertOptions(column_types=posts_schema, strings_can_be_null=True),
).cast(posts_schema)

(Nótese el poco tiempo que tarda en leer el fichero del disco, aunque ocupa casi 1 GB. Esto sucede porque internamente PyArrow está optimizado para leer sólo bloques los bloques de datos necesarios y planificar la lectura del resto de forma perezosa (cuando se necesita). Además, al utilizar un sistema basado en columnas, sólo lee las columnas que realmente necesita para realizar la operación solicitada.)

In [34]:
posts_arrow

pyarrow.Table
Id: int64 not null
AcceptedAnswerId: int64
AnswerCount: int32
Body: string
ClosedDate: timestamp[ms]
CommentCount: int32
CommunityOwnedDate: timestamp[ms]
ContentLicense: string
CreationDate: timestamp[ms]
FavoriteCount: int32
LastActivityDate: timestamp[ms]
LastEditDate: timestamp[ms]
LastEditorDisplayName: string
LastEditorUserId: int64
OwnerDisplayName: string
OwnerUserId: int64
ParentId: int64
PostTypeId: int64 not null
Score: int32
Tags: string
Title: string
ViewCount: int64
----
Id: [[1,2,3,4,5,...,790,791,792,793,794],[795,796,797,798,800,...,1628,1629,1630,1631,1632],...,[608184,608185,608186,608188,608189,...,608716,608717,608718,608720,608721],[608724,608725,608726,608727,608728,...,609180,609181,609182,609183,609184]]
AcceptedAnswerId: [[2,null,9,null,208,...,null,null,null,944,null],[null,null,null,null,null,...,null,null,1646,null,null],...,[null,null,null,null,null,...,null,null,null,null,null],[null,null,null,null,null,...,null,null,null,null,null]]
AnswerC

PyArrow ha significado un avance importante en varios aspectos. Un modelo de memoria basado en columnas muy eficiente en cualquier lenguaje de programación, que permite además utilizar de forma nativa ficheros de intercambio de datos avanzados como Parquet, que veremos a continuación, mucho más interesantes y eficientes que CSV, al incluir, por ejemplo, compresión y codificación de datos, y esquemas.

## Uso de Parquet

![Parquet](https://upload.wikimedia.org/wikipedia/commons/4/47/Apache_Parquet_logo.svg)


El formato Parquet (https://parquet.apache.org) se ha popularizado recientemente con el uso de fuentes de datos en Internet. En general supone una mejora en todos los aspectos con respecto a CSV y en otros con respecto a JSON y JSON lines.

En general, Parquet es un formato de almacenamiento de datos de columnas, que es muy eficiente en términos de espacio y tiempo de acceso. Es un formato binario, pero que se puede leer en muchos lenguajes de programación. Además, permite compresión de datos, lo que lo hace eficiente en tiempo y en espacio.

El formato interno del fichero se describe por encima en la siguiente imagen:

[Parquet](https://raw.githubusercontent.com/dsevilla/bd2-public/25-26/misc/img/parquet.gif)

![Parquet2](https://raw.githubusercontent.com/dsevilla/bd2-public/25-26/misc/img/parquet2.webp)

El formato Parquet incluye, además de los datos, el esquema de los mismos, lo que hace que se pueda leer sin dar lugar a errores. Esto soluciona el problema que nos encontramos en CSV y JSON, que no incluyen el esquema de los datos.

Es incluso fomentado por el Gobierno de España para publicación de los datos: https://datos.gob.es/es/blog/por-que-deberias-de-usar-ficheros-parquet-si-procesas-muchos-datos

En Python también se puede leer con la biblioteca `pyarrow` (https://arrow.apache.org/docs/python/parquet.html).


La escritura de ficheros Parquet se puede hacer de varias formas, aunque por debajo, como dijimos todos utilizan la biblioteca `pyarrow`. Aquí cómo la soporta `pandas`:

```python
# Write the df dataframe to parquet file
df: pd.DataFrame = pd.read_csv('Posts.csv')
df.to_parquet('Posts.parquet', compression='gzip')
```

In [35]:
!ls -lh Posts.*

-rw-r--r-- 1 root root  801M Feb  5 09:06 Posts.csv
-rw-r--r-- 1 root root  1.1G Feb  5 09:07 Posts.json
-rw-r--r-- 1 root root  957M Feb  5 09:09 Posts.jsonl
-rw-r--r-- 1 1000 users 983M Dec  4  2023 Posts.xml


Pero nosotros lo haremos directamente con `pyarrow`, ya que hemos leído antes el CSV en una tabla de pyarrow que tiene además el esquema perfectamente definido:

In [36]:
import pyarrow.parquet as pq

# Escribe la tabla `posts_arrow` en un fichero Parquet comprimido con zstd
pq.write_table(posts_arrow, "Posts.parquet", compression="zstd")

In [37]:
!ls -lh Posts.*

-rw-r--r-- 1 root root  801M Feb  5 09:06 Posts.csv
-rw-r--r-- 1 root root  1.1G Feb  5 09:07 Posts.json
-rw-r--r-- 1 root root  957M Feb  5 09:09 Posts.jsonl
-rw-r--r-- 1 root root  230M Feb  5 09:09 Posts.parquet
-rw-r--r-- 1 1000 users 983M Dec  4  2023 Posts.xml


Vamos a leer ahora el fichero generado con PyArrow. Por supuesto guarda todos los datos, pero también el esquema de forma exacta, ¡incluyendo los metadatos!

In [38]:
# Lee el fichero Parquet Posts.parquet usando pyarrow, y genera un dataframe de pandas
arrow_posts: pa.Table = pq.read_table("Posts.parquet")
arrow_posts.schema

Id: int64 not null
  -- field metadata --
  primary key: 'true'
  description: 'Unique identifier for the post'
AcceptedAnswerId: int64
  -- field metadata --
  description: 'Unique identifier for the accepted answer'
  foreign key: 'Posts.Id'
AnswerCount: int32
  -- field metadata --
  description: 'Number of answers to the post'
  default: '0'
Body: string
  -- field metadata --
  description: 'Body content of the post (compressed, binary)'
ClosedDate: timestamp[ms]
  -- field metadata --
  description: 'Date the post was closed'
CommentCount: int32
  -- field metadata --
  description: 'Number of comments on the post'
  default: '0'
CommunityOwnedDate: timestamp[ms]
  -- field metadata --
  description: 'Date the post was community owned'
ContentLicense: string
  -- field metadata --
  description: 'License for the content'
CreationDate: timestamp[ms]
  -- field metadata --
  description: 'Date the post was created'
FavoriteCount: int32
  -- field metadata --
  description: 'Number 

In [39]:
from pandas import DataFrame

df_posts: DataFrame = arrow_posts.to_pandas(split_blocks=True, self_destruct=True)

In [40]:
df_posts.dtypes

Id                                int64
AcceptedAnswerId                float64
AnswerCount                     float64
Body                             object
ClosedDate               datetime64[ms]
CommentCount                      int32
CommunityOwnedDate       datetime64[ms]
ContentLicense                   object
CreationDate             datetime64[ms]
FavoriteCount                   float64
LastActivityDate         datetime64[ms]
LastEditDate             datetime64[ms]
LastEditorDisplayName            object
LastEditorUserId                float64
OwnerDisplayName                 object
OwnerUserId                     float64
ParentId                        float64
PostTypeId                        int64
Score                             int32
Tags                             object
Title                            object
ViewCount                       float64
dtype: object

In [41]:
df_posts.head()

,Id,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,ContentLicense,CreationDate,FavoriteCount,...,LastEditorDisplayName,LastEditorUserId,OwnerDisplayName,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount
0,1,2.0,1.0,<p>Estoy creando un servicio usando <em>ASP.NE...,NaT,2,NaT,CC BY-SA 4.0,2015-10-29 15:56:52.933,NaN,...,user13558,NaN,None,23.0,NaN,1,40,<.net><asp.net-web-api><asp.net>,La creación manual de un alias de ruta con un ...,780.0
1,2,NaN,NaN,<p>He encontrado la solución.</p><br/><br/><p>...,NaT,2,NaT,CC BY-SA 3.0,2015-10-29 19:14:23.673,NaN,...,None,NaN,None,23.0,1.0,2,31,None,None,NaN
2,3,9.0,3.0,"<p>Luego de ver cierto código, me he dado cuen...",NaT,1,NaT,CC BY-SA 3.0,2015-10-29 23:54:31.947,NaN,...,None,20.0,None,21.0,NaN,1,20,<delphi>,¿Es igual utilizar .AsString que .Text para ob...,1035.0
3,4,NaN,NaN,"<p><code>.AsString</code> devuelve el mismo ""<...",NaT,0,NaT,CC BY-SA 3.0,2015-10-30 00:45:47.640,NaN,...,None,25.0,None,24.0,3.0,2,6,None,None,NaN
4,5,208.0,7.0,<p>¿Cuál es la forma más eficiente de separar ...,NaT,1,NaT,CC BY-SA 3.0,2015-10-30 01:15:27.267,NaN,...,None,729.0,None,24.0,NaN,1,37,<c++><string>,¿Cómo separar las palabras que contiene un str...,37867.0


También se puede guardar el esquema en un fichero serializado para usarlo posteriormente:

In [42]:
# Serialize the schema to a binary format
serialized_schema: bytes = posts_schema.serialize()

# Save the serialized schema to a file
with open("Posts.pb", "wb") as f:
    f.write(serialized_schema)

# Read the serialized schema from the file
with open("Posts.pb", "rb") as f:
    read_schema: bytes = f.read()

# Deserialize the schema
deserialized_schema: pa.Schema = pa.ipc.read_schema(pa.BufferReader(read_schema))

# Verify the schema
print(deserialized_schema)

Id: int64 not null
  -- field metadata --
  primary key: 'true'
  description: 'Unique identifier for the post'
AcceptedAnswerId: int64
  -- field metadata --
  description: 'Unique identifier for the accepted answer'
  foreign key: 'Posts.Id'
AnswerCount: int32
  -- field metadata --
  description: 'Number of answers to the post'
  default: '0'
Body: string
  -- field metadata --
  description: 'Body content of the post (compressed, binary)'
ClosedDate: timestamp[ms]
  -- field metadata --
  description: 'Date the post was closed'
CommentCount: int32
  -- field metadata --
  description: 'Number of comments on the post'
  default: '0'
CommunityOwnedDate: timestamp[ms]
  -- field metadata --
  description: 'Date the post was community owned'
ContentLicense: string
  -- field metadata --
  description: 'License for the content'
CreationDate: timestamp[ms]
  -- field metadata --
  description: 'Date the post was created'
FavoriteCount: int32
  -- field metadata --
  description: 'Number 

El formato Parquet también permite particionado de los datos, lo que permite mejorar el rendimiento en la lectura de los mismos. Esto es especialmente útil cuando se trabaja con grandes volúmenes de datos. Vamos a crear en el dataframe una columna year con el año de la fecha de la pregunta. Después, guardaremos el dataframe en formato Parquet particionado por año.


In [43]:
import pyarrow.compute as pc

posts_arrow: pa.Table = posts_arrow.append_column("year", pc.year(posts_arrow["CreationDate"]))

# Equivalente en Pandas:
# df['year'] = pd.to_datetime(df['CreationDate'], errors='coerce').dt.year

Y escribimos el dataframe en formato Parquet particionado por año. Esto nos permitirá leer los datos más rápidamente cuando necesitemos acceder a un año concreto.

In [44]:
pq.write_to_dataset(
    posts_arrow, "Posts_by_year.parquet", partition_cols=["year"], compression="zstd"
)

# Equivalente en Pandas:
# df.to_parquet('Posts_by_year.parquet', partition_cols=['year'], compression='gzip')

In [45]:
!ls -lhR Posts_by_year.*

Posts_by_year.parquet:
total 36K
drwxr-xr-x 2 root root 4.0K Feb  5 09:09 'year=2015'
drwxr-xr-x 2 root root 4.0K Feb  5 09:09 'year=2016'
drwxr-xr-x 2 root root 4.0K Feb  5 09:09 'year=2017'
drwxr-xr-x 2 root root 4.0K Feb  5 09:09 'year=2018'
drwxr-xr-x 2 root root 4.0K Feb  5 09:09 'year=2019'
drwxr-xr-x 2 root root 4.0K Feb  5 09:09 'year=2020'
drwxr-xr-x 2 root root 4.0K Feb  5 09:09 'year=2021'
drwxr-xr-x 2 root root 4.0K Feb  5 09:09 'year=2022'
drwxr-xr-x 2 root root 4.0K Feb  5 09:09 'year=2023'

'Posts_by_year.parquet/year=2015':
total 812K
-rw-r--r-- 1 root root 810K Feb  5 09:09 c65f17b3025c4433bb14ab0ff998afa7-0.parquet

'Posts_by_year.parquet/year=2016':
total 16M
-rw-r--r-- 1 root root 16M Feb  5 09:09 c65f17b3025c4433bb14ab0ff998afa7-0.parquet

'Posts_by_year.parquet/year=2017':
total 35M
-rw-r--r-- 1 root root 35M Feb  5 09:09 c65f17b3025c4433bb14ab0ff998afa7-0.parquet

'Posts_by_year.parquet/year=2018':
total 39M
-rw-r--r-- 1 root root 39M Feb  5 09:09 c65f17b3025c443